In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import collections

from sklearn.preprocessing import LabelEncoder

import scipy.stats as stats
from scipy.stats import wilcoxon

import copy

In [2]:
log = pd.read_csv("eventsWithPhases.csv")

In [4]:
#create log with Case ID based on currentQuestion + participant

#change data type of currentQuestion from int to str
log = log.astype({'currentQuestion': str})

#combine two columns
log['case_id'] = pd.factorize(log.participant+log.currentQuestion)[0]

In [5]:
# Define conversion function
def convert_ms_to_date(milliseconds):
    date_obj = datetime.fromtimestamp(milliseconds / 1000.0)
    date_string = date_obj.strftime('%Y-%m-%d %H:%M:%S.%f')
    return date_string

# Apply conversion function to 'milliseconds' column
log['fixation_start'] = log['Fixation Start'].apply(convert_ms_to_date)
log['fixation_end'] = log['Fixation End'].apply(convert_ms_to_date)

In [8]:
# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'tabName_element' column
log['activity'] = le.fit_transform(log['tabName_element'])

In [10]:
### log_global_search

In [11]:
#there are 614 cases, although 46 participants * 14 questions = 616 --> 2 cases are missing 
#print(len(log.case_id.unique()))

In [12]:
log

,Number,participant,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,SacID,...,Type3,Relevant elements labels,Relevant elements count,Relevant elements names,relevant,Phase,case_id,fixation_start,fixation_end,activity
0,0,KP1-no,1101.0,1301.2509,133.2523,5.715801e+06,5.715884e+06,83.4080,0.1208,NaN,...,Exclusiveness,['Send detailed information on final destinati...,2,"['Activity_1fk1rvd', 'Activity_0v934ow']",0,search,0,1970-01-01 02:35:15.800704,1970-01-01 02:35:15.884111,82
1,1,KP1-no,1102.0,577.2880,-37.4236,5.716709e+06,5.716834e+06,124.9670,0.3481,NaN,...,Exclusiveness,['Send detailed information on final destinati...,2,"['Activity_1fk1rvd', 'Activity_0v934ow']",0,search,0,1970-01-01 02:35:16.708983,1970-01-01 02:35:16.833950,82
2,2,KP1-no,1103.0,453.3125,-42.5000,5.716884e+06,5.716951e+06,66.6000,0.2288,NaN,...,Exclusiveness,['Send detailed information on final destinati...,2,"['Activity_1fk1rvd', 'Activity_0v934ow']",0,search,0,1970-01-01 02:35:16.883977,1970-01-01 02:35:16.950577,82
3,3,KP1-no,1104.0,579.9746,-80.0503,5.717900e+06,5.717984e+06,83.3050,1.5229,NaN,...,Exclusiveness,['Send detailed information on final destinati...,2,"['Activity_1fk1rvd', 'Activity_0v934ow']",0,search,0,1970-01-01 02:35:17.900487,1970-01-01 02:35:17.983792,82
4,4,KP1-no,1105.0,647.4488,-46.5585,5.718375e+06,5.718484e+06,108.3100,0.2081,NaN,...,Exclusiveness,['Send detailed information on final destinati...,2,"['Activity_1fk1rvd', 'Activity_0v934ow']",0,search,0,1970-01-01 02:35:18.375424,1970-01-01 02:35:18.483734,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174696,174696,SP11-no,5752.0,789.3538,186.0428,9.295871e+06,9.296079e+06,208.2810,0.2594,NaN,...,Data-based Routing,"['Container [in transition]', 'Collect contain...",3,"['DataObjectReference_09inboe', 'Activity_0qpe...",0,NaN,613,1970-01-01 03:34:55.871112,1970-01-01 03:34:56.079393,650
174697,174697,SP11-no,5753.0,1229.2347,118.4895,9.296179e+06,9.296563e+06,383.2750,0.3156,NaN,...,Data-based Routing,"['Container [in transition]', 'Collect contain...",3,"['DataObjectReference_09inboe', 'Activity_0qpe...",0,NaN,613,1970-01-01 03:34:56.179382,1970-01-01 03:34:56.562657,605
174698,174698,SP11-no,5754.0,1282.5789,124.7105,9.296621e+06,9.296779e+06,158.3300,0.2466,NaN,...,Data-based Routing,"['Container [in transition]', 'Collect contain...",3,"['DataObjectReference_09inboe', 'Activity_0qpe...",0,NaN,613,1970-01-01 03:34:56.620999,1970-01-01 03:34:56.779329,605
174699,174699,SP11-no,5755.0,911.5417,199.0000,9.296846e+06,9.296946e+06,100.0035,0.1728,NaN,...,Data-based Routing,"['Container [in transition]', 'Collect contain...",3,"['DataObjectReference_09inboe', 'Activity_0qpe...",0,NaN,613,1970-01-01 03:34:56.845958,1970-01-01 03:34:56.945961,650


In [13]:
#Select only Control-flow questions
log_select = log[['case_id', 'fixation_start', 'activity', 'Phase', 'Type1', 'Type2', 'Type3', 'Fixation Duration', 'participant']]
log_tasks = log_select.loc[log_select['Type2'] == 'Control-flow']
log_tasks['taskType'] = log_tasks['Type1'] + log_tasks['Type3']

log_global_search = log_tasks.loc[(log_tasks['Type1'] == 'Global') & (log_tasks['Phase'] == 'search')]
log_global_inference = log_tasks.loc[(log_tasks['Type1'] == 'Global') & (log_tasks['Phase'] == 'inference')]
log_local_search = log_tasks.loc[(log_tasks['Type1'] == 'Local') & (log_tasks['Phase'] == 'search')]
log_local_inference = log_tasks.loc[(log_tasks['Type1'] == 'Local') & (log_tasks['Phase'] == 'inference')]


#Create trace logs
logVar_global_search = log_global_search.groupby(['case_id'])['activity'].apply(list).reset_index()
logVar_global_inference = log_global_inference.groupby(['case_id'])['activity'].apply(list).reset_index()
logVar_local_search = log_local_search.groupby(['case_id'])['activity'].apply(list).reset_index()
logVar_local_inference = log_local_inference.groupby(['case_id'])['activity'].apply(list).reset_index()

C:\Users\la1949\AppData\Local\Temp\ipykernel_11236\3806467676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_tasks['taskType'] = log_tasks['Type1'] + log_tasks['Type3']


In [58]:
#log['activity'] = le.fit_transform(log['tabName_element'])
#log_select = log[['case_id', 'fixation_start', 'activity', 'Phase', 'Type1', 'Type2', 'Type3', 'Fixation Duration', 'participant']]
#log_tasks = log_select.loc[log_select['Type2'] == 'Control-flow']
#log_tasks['taskType'] = log_tasks['Type1'] + log_tasks['Type3']

log_global = log_tasks.loc[(log_tasks['Type1'] == 'Global')]
log_local = log_tasks.loc[(log_tasks['Type1'] == 'Local')]


logVar_global = log_global.groupby(['case_id'])['activity'].apply(list).reset_index()
logVar_local = log_local.groupby(['case_id'])['activity'].apply(list).reset_index()

In [14]:
log_global_search_participant = log_global_search.groupby(['case_id'])['participant'].apply(list).reset_index()
log_global_search_participant['participant'] = log_global_search_participant['participant'].apply(lambda x: set(x))
log_global_search_participant['participant'] = log_global_search_participant['participant'].apply(lambda x: ''.join(x))
logVar_global_search = pd.merge(logVar_global_search, log_global_search_participant, on='case_id')


log_global_inference_participant = log_global_inference.groupby(['case_id'])['participant'].apply(list).reset_index()
log_global_inference_participant['participant'] = log_global_inference_participant['participant'].apply(lambda x: set(x))
log_global_inference_participant['participant'] = log_global_inference_participant['participant'].apply(lambda x: ''.join(x))
logVar_global_inference = pd.merge(logVar_global_inference, log_global_inference_participant, on='case_id')

log_local_search_participant = log_local_search.groupby(['case_id'])['participant'].apply(list).reset_index()
log_local_search_participant['participant'] = log_local_search_participant['participant'].apply(lambda x: set(x))
log_local_search_participant['participant'] = log_local_search_participant['participant'].apply(lambda x: ''.join(x))
logVar_local_search = pd.merge(logVar_local_search, log_local_search_participant, on='case_id')

log_local_inference_participant = log_local_inference.groupby(['case_id'])['participant'].apply(list).reset_index()
log_local_inference_participant['participant'] = log_local_inference_participant['participant'].apply(lambda x: set(x))
log_local_inference_participant['participant'] = log_local_inference_participant['participant'].apply(lambda x: ''.join(x))
logVar_local_inference = pd.merge(logVar_local_inference, log_local_inference_participant, on='case_id')


In [60]:
log_global_participant = log_global.groupby(['case_id'])['participant'].apply(list).reset_index()
log_global_participant['participant'] = log_global_participant['participant'].apply(lambda x: set(x))
log_global_participant['participant'] = log_global_participant['participant'].apply(lambda x: ''.join(x))
logVar_global = pd.merge(logVar_global, log_global_participant, on='case_id')

log_local_participant = log_local.groupby(['case_id'])['participant'].apply(list).reset_index()
log_local_participant['participant'] = log_local_participant['participant'].apply(lambda x: set(x))
log_local_participant['participant'] = log_local_participant['participant'].apply(lambda x: ''.join(x))
logVar_local = pd.merge(logVar_local, log_local_participant, on='case_id')

In [16]:
log_tasks.Phase.unique() # --> why nan?

array(['search', 'inference', nan], dtype=object)

In [17]:
# Calculate Entropy

def summary_statistic(window):
    counter = collections.Counter(window)  # count the occurrences of each element in the window
    freqs = np.array(list(counter.values()))  # extract the frequency counts and store them as a numpy array
    freqs = freqs / freqs.sum()  # normalize the frequencies to obtain a probability distribution
    entropy = -np.sum(freqs * np.log(freqs)) / np.log(2) # compute the entropy of the probability distribution
    return entropy


def avg_trace_entropy(log):
    traces = log["trace_variant"]
    for i in traces:
        #print(i)
        print(summary_statistic(i))

#Example
#a = ["A", "A", "A", "A", "B"]        
#summary_statistic(a)

In [18]:
# Add entropy to df

logVar_global_search['trace_entropy'] = logVar_global_search['activity'].apply(summary_statistic)
logVar_global_inference['trace_entropy'] = logVar_global_inference['activity'].apply(summary_statistic)
logVar_local_search['trace_entropy'] = logVar_local_search['activity'].apply(summary_statistic)
logVar_local_inference['trace_entropy'] = logVar_local_inference['activity'].apply(summary_statistic)

In [19]:
logVar_global_search

,case_id,activity,participant,trace_entropy
0,5,"[90, 82, 183, 82, 82, 82, 82, 82, 183, 77, 75,...",KP1-no,5.137653
1,9,"[82, 82, 82, 110, 82, 179, 82, 82, 82, 82, 82,...",KP1-no,2.981627
2,18,"[89, 50, 56, 181, 181, 181, 181, 181, 181, 181...",KP10-no,6.390246
3,19,"[93, 50, 183, 59, 183, 183, 183, 183, 183, 183...",KP10-no,4.174346
4,23,"[88, 110, 179, 179, 179, 179, 179, 179, 112, 7...",KP10-no,5.793971
...,...,...,...,...
151,592,"[82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 8...",SP9-no,4.140865
152,597,"[179, 82, 179, 82, 82, 82, 82, 82, 82, 82, 82,...",SP9-no,5.471406
153,598,"[82, 82, 82, 82, 82, 82, 82, 50, 50, 3, 27, 32...",SP9-no,5.631224
154,605,"[50, 181, 181, 181, 82, 181, 56, 181, 113, 181...",SP11-no,5.000970


In [20]:
#logVar_global_search[['trace_entropy']
logVar_global_search_agg = logVar_global_search.groupby(['participant']).agg({'trace_entropy':'mean'})
logVar_global_inference_agg = logVar_global_inference.groupby(['participant']).agg({'trace_entropy':'mean'})
logVar_local_search_agg = logVar_local_search.groupby(['participant']).agg({'trace_entropy':'mean'})
logVar_local_inference_agg = logVar_local_inference.groupby(['participant']).agg({'trace_entropy':'mean'})

In [44]:
logVar_global_search

,case_id,activity,participant,trace_entropy
0,5,"[90, 82, 183, 82, 82, 82, 82, 82, 183, 77, 75,...",KP1-no,5.137653
1,9,"[82, 82, 82, 110, 82, 179, 82, 82, 82, 82, 82,...",KP1-no,2.981627
2,18,"[89, 50, 56, 181, 181, 181, 181, 181, 181, 181...",KP10-no,6.390246
3,19,"[93, 50, 183, 59, 183, 183, 183, 183, 183, 183...",KP10-no,4.174346
4,23,"[88, 110, 179, 179, 179, 179, 179, 179, 112, 7...",KP10-no,5.793971
...,...,...,...,...
151,592,"[82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 8...",SP9-no,4.140865
152,597,"[179, 82, 179, 82, 82, 82, 82, 82, 82, 82, 82,...",SP9-no,5.471406
153,598,"[82, 82, 82, 82, 82, 82, 82, 50, 50, 3, 27, 32...",SP9-no,5.631224
154,605,"[50, 181, 181, 181, 82, 181, 56, 181, 113, 181...",SP11-no,5.000970


In [21]:
print('global search:', logVar_global_search_agg.mean())
print('global inference:', logVar_global_inference_agg.mean())
print('local search:', logVar_local_search_agg.mean())
print('local inference:', logVar_local_inference_agg.mean())

global search: trace_entropy    4.650152
dtype: float64
global inference: trace_entropy    4.708551
dtype: float64
local search: trace_entropy    3.939616
dtype: float64
local inference: trace_entropy    3.153731
dtype: float64


In [23]:
#global
stats.wilcoxon(logVar_global_search_agg['trace_entropy'], logVar_global_inference_agg['trace_entropy'],alternative='greater')

WilcoxonResult(statistic=462.0, pvalue=0.6506866179365716)

In [24]:
#local
stats.wilcoxon(logVar_local_search_agg['trace_entropy'], logVar_local_inference_agg['trace_entropy'],alternative='greater')

WilcoxonResult(statistic=930.0, pvalue=5.761705779150361e-09)

In [25]:
#search
stats.wilcoxon(logVar_local_search_agg['trace_entropy'], logVar_global_search_agg['trace_entropy'],alternative='less')

WilcoxonResult(statistic=67.0, pvalue=1.2301768492761767e-08)

In [26]:
#inference
stats.wilcoxon(logVar_local_inference_agg['trace_entropy'], logVar_global_inference_agg['trace_entropy'],alternative='less')

WilcoxonResult(statistic=9.0, pvalue=1.8758328224066645e-12)